In [1]:
from coptpy import *
import numpy as np

env =Envr()
model=env.createModel(name="SDP")

rows=[0,1,1,2,2]
cols=[0,0,1,1,2]
vals=[2,1,2,1,2]

C=model.addSparseMat(3,rows,cols,vals)

A1=model.addEyeMat(3)

A2=model.addOnesMat(3)

x0 = model.addVar(lb=0.0, ub=COPT.INFINITY, name="x0")
x1 = model.addVar(lb=0.0, ub=COPT.INFINITY, name="x1")
x2 = model.addVar(lb=0.0, ub=COPT.INFINITY, name="x2")
X=model.addPsdVar(3,'X')

model.addConstr(x0 + x1 + x2<=0.9,name='lc')
model.addConstr(A1*X<=0.8,name='PSD_1')
model.addConstr(A2*X + x1 +x2 ==0.6,name='PSD_2')
cone=model.addCone([x0,x1,x2],COPT.CONE_QUAD)
model.setObjective(C*X+x0 ,sense=COPT.MINIMIZE)

model.solve()

Cardinal Optimizer v7.2.9. Build date Jun 20 2025
Copyright Cardinal Operations 2025. All Rights Reserved

Model fingerprint: 8063cda6

Using Cardinal Optimizer v7.2.9 on Windows
Hardware has 6 cores and 12 threads. Using instruction set X86_NATIVE (1)
Minimizing an SDP problem

The original problem has:
    1 rows, 3 columns and 3 non-zero elements
    1 PSD columns
    1 PSD objective elements
    2 PSD constraints
    1 cones

Using Primal-Dual Method

Starting barrier solver using 6 CPU threads

Problem info:
Range of matrix coefficients:    [5e-01,1e+00]
Range of rhs coefficients:       [3e-01,9e-01]
Range of bound coefficients:     [0e+00,0e+00]
Range of cost coefficients:      [3e-01,5e-01]

Factor info:
Number of dense columns:         0
Number of matrix entries:        5.000e+00
Number of factor entries:        5.000e+00
Number of factor flops:          9.000e+00

Iter       Primal.Obj         Dual.Obj      Compl  Primal.Inf  Dual.Inf    Time
   0  +7.41421356e+00  +0.00000000

In [2]:
# map: num --> model status 
status_map = {
    0:"COPT.UNSTARTED",
    1:"COPT.OPTIMAL",
    2:"COPT.INFEASIBLE",
    3:"COPT.UNBOUNDED",
    4:"COPT.INF_OR_UNB",
    5:"COPT.NUMERICAL",
    6:"COPT.NODELIMIT",
    7:"COPT.TIMEOUT",
    8:"COPT.UNFINISHED",
    9:"COPT.INTERRUPTED"
}

In [3]:
# Output solution
if model.status == COPT.OPTIMAL:
    print("Model Status:{0}".format(status_map[model.status]))
    print("\nOptimal objective value: {0}".format(model.objval))
    print("")
    # Obtain Non-PSD variable solution
    print("*"*15,"Non-PSD variable solution","*"*15)
    linvars = model.getVars()
    for linvar in linvars:
        print("Non-PSD variable {0}, value = {1}".format(linvar.name, linvar.x))
    print("")
    # Obtain PSD variable solution
    print("*"*17,"PSD variable solution","*"*17)
    psdvars = model.getPsdVars()
    for psdvar in psdvars:
        value = np.reshape(psdvar.x, (psdvar.dim, psdvar.dim))
        print("PSD variable {0}, value = \n{1}".format(psdvar.name, value))   
else:
    print("No SDP solution")
    print("Model Status:{0}".format(status_map[model.status]))

Model Status:COPT.OPTIMAL

Optimal objective value: 0.44558441206097943

*************** Non-PSD variable solution ***************
Non-PSD variable x0, value = 0.37279220556483084
Non-PSD variable x1, value = 0.26360389650499405
Non-PSD variable x2, value = 0.26360389650499405

***************** PSD variable solution *****************
PSD variable X, value = 
[[1.81979526e-02 9.90212114e-08 1.81979525e-02]
 [9.90212114e-08 8.86723157e-11 9.90212114e-08]
 [1.81979525e-02 9.90212114e-08 1.81979526e-02]]
